# Image Transformations
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Denoising](#noise)
* [Thresholding: separate foreground from background](#threshold)
* [Non-uniform illumination correction](#bgcorrect)
* [Morphological operations](#morphology)

## Introduction
<a id="intro"></a> 
Once we can handle images of any OME-supported format in Python, we can apply a number of transformations to correct brightness, contrast, noise levels, etc


After the images have been loaded and inspected, they can be transformed. To name a few image transformations:
+ adjust brightness and contrast
+ rescale image
+ remove background
+ denoise / introduce Gaussian blur
+ separate foreground from background (threshold)

Here we will talk about the most basic image transformations such as contrast adjustment and rescaling (see 'Advanced Image Processing' notebook for the detailed treatment of image transformations).

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

Load an image:

In [ ]:
from transform.basic import read_tiff
img_ho = read_tiff(fname='data/CLL-coculture/r01c02f01-Hoechst.tiff')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_ho, cmap='gray')
plt.axis('off')

To adjust brightness of an image, one may have to apply gamma correction to the image, which simply raises the `
np.array` to the power of `gamma`

In [ ]:
img_ho.dtype

In [ ]:
# if the numpy.array is of dtype 'float'
gamma = 0.3
img_gamma = img_ho ** gamma

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_gamma, cmap='gray')
plt.axis('off')


Perhaps the most trivial image transformation is image inversion (obtaining the "negative" of the image)

In [ ]:
# if the numpy.array is of dtype 'float'
img_neg = 1. - img_gamma

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_neg, cmap='gray')
plt.axis('off')

A useful feature is image rescaling, which can be achieved by `skimage.transform.resize`:

In [ ]:
# dimensions of the transformd image
img_neg.shape

To downsample the image:

In [ ]:
from skimage.transform import resize
img_small = resize(img_neg, (200, 200), anti_aliasing=False)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img_small, cmap='gray')

Consider the downsampled negative image, by downsampling it from the original size of (2160 x 2160) to (200 x 200) pixels we introduced some image artefacts. A common approach to this is anti-aliasing which for example could apply Gaussian blur to the image

## Denoising
<a id="noise"></a> 
Smoothing speckle noise. Talk about Fourier-space analysis for complex noise structure and non-local means denoising.

The simplest "denoising" technique is applying Gaussian blur:

In [ ]:
from skimage.filters import gaussian

img_smooth = gaussian(img_small, sigma=0.55)

In [ ]:
from visualize.plot_static import plot_channels
plot_channels([img_small, img_smooth], nrow=1, ncol=2, 
              cmap='gray',
              titles = ['Downsampled (200x200)', 'Smoothed (200 x 200)'],
             scale_x=6, scale_y=6)

In [ ]:
from transform.basic import write_image
write_image(img_smooth, 'data/img_neg_downsampled.png')

## Thresholding: separate foreground from background
<a id="threshold"></a> 

The best is always to compare the original and transformed image side-by-side:

## Illumination correction
<a id="bgcorrect"></a>
Non-uniform background removal

In [ ]:
from transform.basic import load_imgstack
imgstack = load_imgstack(fname="data/BiTE/Tag2-r04c02f1.tiff")

In [ ]:
mip = np.amax(imgstack, axis=0)

In [ ]:

# split individual color channels and place them in a list
mip_split = [mip[:,:,i] for i in range(mip.shape[2])]
plot_channels(mip_split,
              nrow=1, ncol=4,
              titles=['CD20+', 'Calcein',
                      'Nuclei', 'CD8+'])

Let's check the nuclei first:

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(mip[:,:,2]**0.7, cmap='gray')
plt.axis('off')

## Morphological operations
<a id="morphology"></a> 